In [2]:
%reload_ext autoreload
%autoreload 2
from SFM import SFM
import database
import os, shutil, sys
import pycolmap, platform
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
src = "datasets/Viking"
dest = "colmap_test_out"
db_path = "test.db"

try:
    os.mkdir(dest)
except:
    shutil.rmtree(dest)
    os.mkdir(dest)

In [4]:
test = SFM(src)
test.ft_extract()
matches = test.ft_match()

48.jpg: 100%|██████████| 49/49 [00:00<00:00, 139.09it/s]




FEATURE EXTRACTION....


Image 49: 100%|██████████| 49/49 [00:05<00:00,  8.91it/s]



MATCHING...



(47, 48): 100%|██████████| 48/48 [01:33<00:00,  1.95s/it]                         


Elapsed Time: 93.66186618804932




In [5]:
db = database.COLMAPDatabase.connect(db_path)

In [6]:
db.create_tables()

In [7]:
# add camera and images

cam = db.add_camera(0, 504, 672, np.array([806.4, 252.0, 336.0]))

for file in os.listdir(test.src):
    db.add_image(f"{file}", 1)

db.commit()

In [8]:
# add keypoints and descriptors

kps_per_frame = [np.array([[y.pt[0], y.pt[1], y.size, y.angle] for y in x[0]], dtype=np.float32) for x in test.desc]
desc_per_frame = [x[1] for x in test.desc]

for i, frame_kps in enumerate(kps_per_frame):
    db.add_keypoints(i+1, frame_kps)

for i, frame_desc in enumerate(desc_per_frame):
    db.add_descriptors(i+1, frame_desc)

db.commit()


In [93]:

# np.set_printoptions(threshold=sys.maxsize)
f = db.execute("""select count(*) from matches""")
f.fetchall()
# [np.array([y for y in x[1]]) for x in test.desc]
# [x[1] for x in test.desc]
# print([np.array([y.pt for y in x[0]], dtype=np.float32) for x in test.desc][0].tobytes())
# print('\n\n\n\n\n\n\n')
# print(np.random.rand(2097, 2) * (504, 672))
# [np.array([[y.pt[0], y.pt[1], y.size, y.angle] for y in x[0]], dtype=np.float32) for x in test.desc]

# for i in test.desc:
#     for j in i[0]:
#         print(j.pt, j.size, j.angle)
# test.output_match()

[(238,)]

In [9]:
# add matches

for i in matches.keys():
    if isinstance(i, tuple):
        m = np.array([[x.queryIdx,x.trainIdx] for x in matches[i]], dtype=np.uint32)
        if len(m) > 0:
            db.add_matches(i[0]+1, i[1]+1, m)

db.commit()

In [10]:
db.close()

In [14]:
pycolmap.match_exhaustive(db_path)

{0: Reconstruction(num_reg_images=49, num_cameras=1, num_points3D=7371, num_observations=28363)}


In [ ]:
maps = pycolmap.incremental_mapping(db_path, src, dest)